## Quantitative Momentum Strategy

"Momentum Investing" means investing in the stocks that have increased in price the most. 

In this notebook, we are going to build an investing strategy that selects the 50 stocks with highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. 

### Library Imports

The first thing we need to do is import the open-source software libraries that we will be using in this tutorial. 

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

### Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from api_token import IEX_CLOUD_API_TOKEN

### Making Our First API Call

It's now time to make the first version of momentum screener!

We need to get one year price returns for each stock in the universe. This metric is present in this iex cloud data endpoint: https://iexcloud.io/docs/core/ADVANCED_STATS

In [3]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
data

{'beta': 1.317362224979355,
 'totalCash': 62482000000,
 'currentDebt': 124963000000,
 'revenue': 383933000000,
 'grossProfit': 166816000000,
 'totalRevenue': 383933000000,
 'EBITDA': 123957000000,
 'revenuePerShare': 24.56,
 'revenuePerEmployee': 2611789.12,
 'debtToEquity': 5.558582473371603,
 'profitMargin': 0.2468138972164414,
 'enterpriseValue': 2832505457760,
 'enterpriseValueToRevenue': 7.38,
 'priceToSales': 7.26,
 'priceToBook': 46.22,
 'forwardPERatio': 29.868014323069666,
 'pegRatio': -17.09169730649125,
 'peHigh': 16.67893138319145,
 'peLow': 12.711051914165973,
 'week52highDate': '2023-07-19',
 'week52lowDate': '2023-01-03',
 'putCallRatio': 0.48717312832382687,
 'companyName': 'Apple Inc',
 'marketcap': 2785707457760,
 'week52high': 197.7,
 'week52low': 123.48,
 'week52highSplitAdjustOnly': 198.23,
 'week52highDateSplitAdjustOnly': '2023-07-19',
 'week52lowSplitAdjustOnly': 124.17,
 'week52lowDateSplitAdjustOnly': '2023-01-03',
 'week52change': 0.14193060486816989,
 'share

### Parsing Our API Call

This API call has all the information we need. Let's parse it to get the one year percent change or returns for one stock. 

In [4]:
data['year1ChangePercent']

0.14193060486816989

### Executing A Batch API Call and Building Our DataFrame

Just like in our first notebook, it's time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called **chunks** that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank dataframe and add our data to the dataframe one by one.

In [6]:
final_dataframe = pd.DataFrame(columns= my_columns)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [7]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    symbol_str_lst = symbol_string.split(',')
    len_ss = len(symbol_str_lst)
    for i in range(len_ss):
        final_dataframe = pd.concat(
            [final_dataframe,
             pd.Series([
                 symbol_str_lst[i],
                 data[i]['latestPrice'],
                 data[i+len_ss]['year1ChangePercent'],
                 'N/A'
             ], index = my_columns).to_frame().transpose()],
            ignore_index=True)

In [8]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,113.99,-0.166151,N/A
1,AAL,13.98,-0.014799,N/A
2,AAP,64.61,-0.629478,N/A
3,AAPL,178.18,0.141931,N/A
4,ABBV,149.02,0.09649,N/A
...,...,...,...,...
500,YUM,126.53,0.110276,N/A
501,ZBH,120.65,0.061853,N/A
502,ZBRA,256.37,-0.167197,N/A
503,ZION,34.92,-0.377491,N/A


### Removing Low-Momentum Stocks

The investment strategy that we are building seeks to identify 50 highest momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We will sort the DataFrame by the stocks' one-year price return and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)

In [10]:
final_dataframe = final_dataframe[:50].reset_index(drop=True)

In [11]:
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,N/A
1,NVDA,455.72,2.169521,N/A
2,FTI,21.03,1.332701,N/A
3,RCL,97.35,1.060754,N/A
4,PHM,80.72,0.956531,N/A


### Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we are going to make is wrapping this functionality inside a function, since we will be using it again later.

In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000


1000000


In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
position_size

20000.0

In [14]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Price'])

In [15]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,250
1,NVDA,455.72,2.169521,43
2,FTI,21.03,1.332701,951
3,RCL,97.35,1.060754,205
4,PHM,80.72,0.956531,247
5,GE,111.72,0.924714,179
6,NFLX,442.8,0.895791,45
7,LLY,586.46,0.865915,34
8,AVGO,857.55,0.699111,23
9,ORCL,126.32,0.691848,158


### Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks.

 - High-quality momentum stocks show "slow and steady" outperformance over long periods of time
 - Low quality momentum stocks might not show any momentum for a long time, and then surge upwards. 

The reason why high quality momentum stocks are preferred is because low-quality momentum can often be caused by short term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high quality momentum, we are going to build a strategy that selects stocks from the highest percentiles of :

 - 1-month price returns
 - 3-month price returns
 - 6-month price returns 
 - 1-year price returns 
 
 The term **hqm** will be used ahead for **high quality momentum**.

In [16]:
hqm_columns = [
    'Ticker',
    'Price',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'Number of Shares to Buy',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score


In [17]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    symbol_str_lst = symbol_string.split(',')
    len_ss = len(symbol_str_lst)
    for i in range(len_ss):
        hqm_dataframe = pd.concat(
            [hqm_dataframe,
             pd.Series([
                 symbol_str_lst[i],
                 data[i]['latestPrice'],
                 data[i+len_ss]['year1ChangePercent'],
                 'N/A',
                 data[i+len_ss]['month6ChangePercent'],
                 'N/A',
                 data[i+len_ss]['month3ChangePercent'],
                 'N/A',
                 data[i+len_ss]['month1ChangePercent'],
                 'N/A',
                 'N/A',
                 'N/A'
             ], index = hqm_columns).to_frame().transpose()],
            ignore_index=True)
        # to handle null values that cause error downstream, we fill nas with -inf so that their percentiles are lowest, hence least likely for investment
        hqm_dataframe.fillna(-np.inf, inplace=True)

In [18]:
hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,A,113.99,-0.166151,N/A,-0.176609,N/A,-0.013665,N/A,-0.106871,N/A,N/A,N/A
1,AAL,13.98,-0.014799,N/A,-0.119647,N/A,-0.102119,N/A,-0.112944,N/A,N/A,N/A
2,AAP,64.61,-0.629478,N/A,-0.494334,N/A,0.019804,N/A,-0.092556,N/A,N/A,N/A
3,AAPL,178.18,0.141931,N/A,0.188051,N/A,-0.012702,N/A,0.002646,N/A,N/A,N/A
4,ABBV,149.02,0.09649,N/A,0.031952,N/A,0.090404,N/A,-0.011607,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.53,0.110276,N/A,0.013217,N/A,-0.047341,N/A,-0.046198,N/A,N/A,N/A
501,ZBH,120.65,0.061853,N/A,-0.01037,N/A,-0.111458,N/A,-0.036419,N/A,N/A,N/A
502,ZBRA,256.37,-0.167197,N/A,-0.13654,N/A,-0.066591,N/A,-0.005894,N/A,N/A,N/A
503,ZION,34.92,-0.377491,N/A,-0.130909,N/A,0.144325,N/A,-0.055471,N/A,N/A,N/A


### Calculating Momentum Percentiles 

We now need to calculate momentum percentile scores for every stock in our dataframe. Let's see how to do this:

In [19]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
        
hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,A,113.99,-0.166151,0.20396,-0.176609,0.112871,-0.013665,0.425743,-0.106871,0.10297,N/A,N/A
1,AAL,13.98,-0.014799,0.429703,-0.119647,0.176238,-0.102119,0.142574,-0.112944,0.091089,N/A,N/A
2,AAP,64.61,-0.629478,0.037624,-0.494334,0.037624,0.019804,0.520792,-0.092556,0.140594,N/A,N/A
3,AAPL,178.18,0.141931,0.681188,0.188051,0.815842,-0.012702,0.431683,0.002646,0.710891,N/A,N/A
4,ABBV,149.02,0.09649,0.605941,0.031952,0.518812,0.090404,0.742574,-0.011607,0.615842,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.53,0.110276,0.635644,0.013217,0.473267,-0.047341,0.267327,-0.046198,0.336634,N/A,N/A
501,ZBH,120.65,0.061853,0.556436,-0.01037,0.407921,-0.111458,0.128713,-0.036419,0.415842,N/A,N/A
502,ZBRA,256.37,-0.167197,0.20198,-0.13654,0.154455,-0.066591,0.219802,-0.005894,0.665347,N/A,N/A
503,ZION,34.92,-0.377491,0.067327,-0.130909,0.162376,0.144325,0.869307,-0.055471,0.275248,N/A,N/A


### Calculating the HQM Score

We will now calculate our **HQM Score**, which is the high quality momentum score that we will use to filter for stocks in this investment strategy. 

The **HQM Score** will be the arithmetic mean of the 4 momentum percentile scores that we calculated in last cell.

To calculate arithmetic mean, we will use the **mean** function from Python's built-in **statistics** module. 

In [20]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,A,113.99,-0.166151,0.20396,-0.176609,0.112871,-0.013665,0.425743,-0.106871,0.10297,N/A,0.211386
1,AAL,13.98,-0.014799,0.429703,-0.119647,0.176238,-0.102119,0.142574,-0.112944,0.091089,N/A,0.209901
2,AAP,64.61,-0.629478,0.037624,-0.494334,0.037624,0.019804,0.520792,-0.092556,0.140594,N/A,0.184158
3,AAPL,178.18,0.141931,0.681188,0.188051,0.815842,-0.012702,0.431683,0.002646,0.710891,N/A,0.659901
4,ABBV,149.02,0.09649,0.605941,0.031952,0.518812,0.090404,0.742574,-0.011607,0.615842,N/A,0.620792
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.53,0.110276,0.635644,0.013217,0.473267,-0.047341,0.267327,-0.046198,0.336634,N/A,0.428218
501,ZBH,120.65,0.061853,0.556436,-0.01037,0.407921,-0.111458,0.128713,-0.036419,0.415842,N/A,0.377228
502,ZBRA,256.37,-0.167197,0.20198,-0.13654,0.154455,-0.066591,0.219802,-0.005894,0.665347,N/A,0.310396
503,ZION,34.92,-0.377491,0.067327,-0.130909,0.162376,0.144325,0.869307,-0.055471,0.275248,N/A,0.343564


### Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks by sorting our dataframe on **HQM Score** column and selecting all the top 50 entries. 

In [21]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50].reset_index(drop=True)
hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,FTI,21.03,1.332701,0.99604,0.431615,0.976238,0.413382,0.99604,0.152967,0.99604,N/A,0.991089
1,LLY,586.46,0.865915,0.986139,0.8597,0.99802,0.317633,0.982178,0.116845,0.988119,N/A,0.988614
2,LB,79.92,2.281287,1.0,0.815831,0.99604,0.215421,0.940594,0.079271,0.960396,N/A,0.974257
3,ETN,236.13,0.65187,0.980198,0.366552,0.958416,0.265719,0.968317,0.08401,0.968317,N/A,0.968812
4,NVDA,455.72,2.169521,0.99802,0.944892,1.0,0.175542,0.908911,0.07101,0.948515,N/A,0.963861
5,ADBE,560.36,0.419423,0.930693,0.656057,0.994059,0.234273,0.946535,0.090661,0.974257,N/A,0.961386
6,ANET,196.47,0.579214,0.972277,0.321695,0.942574,0.208897,0.938614,0.107123,0.980198,N/A,0.958416
7,ORCL,126.32,0.691848,0.982178,0.465341,0.980198,0.153968,0.881188,0.116888,0.990099,N/A,0.958416
8,MPC,155.31,0.590592,0.976238,0.224206,0.861386,0.381277,0.994059,0.083639,0.964356,N/A,0.94901
9,INTC,38.01,0.258618,0.845545,0.45984,0.978218,0.22157,0.942574,0.10881,0.984158,N/A,0.937624


### Calculating the Number of Shares to Buy

We will use the **portfolio_input** function that we created earlier to accept our portfolio size. Then we will use similar logic used in equal_weight strategy to calculate the number of shares to buy for each stock. 

In [22]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [23]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
position_size

20000.0

In [24]:
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])

In [25]:
hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Number of Shares to Buy,HQM Score
0,FTI,21.03,1.332701,0.99604,0.431615,0.976238,0.413382,0.99604,0.152967,0.99604,951,0.991089
1,LLY,586.46,0.865915,0.986139,0.8597,0.99802,0.317633,0.982178,0.116845,0.988119,34,0.988614
2,LB,79.92,2.281287,1.0,0.815831,0.99604,0.215421,0.940594,0.079271,0.960396,250,0.974257
3,ETN,236.13,0.65187,0.980198,0.366552,0.958416,0.265719,0.968317,0.08401,0.968317,84,0.968812
4,NVDA,455.72,2.169521,0.99802,0.944892,1.0,0.175542,0.908911,0.07101,0.948515,43,0.963861
5,ADBE,560.36,0.419423,0.930693,0.656057,0.994059,0.234273,0.946535,0.090661,0.974257,35,0.961386
6,ANET,196.47,0.579214,0.972277,0.321695,0.942574,0.208897,0.938614,0.107123,0.980198,101,0.958416
7,ORCL,126.32,0.691848,0.982178,0.465341,0.980198,0.153968,0.881188,0.116888,0.990099,158,0.958416
8,MPC,155.31,0.590592,0.976238,0.224206,0.861386,0.381277,0.994059,0.083639,0.964356,128,0.94901
9,INTC,38.01,0.258618,0.845545,0.45984,0.978218,0.22157,0.942574,0.10881,0.984158,526,0.937624


### Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.<br>
XlsxWriter is an excellent package and offers tons of customization. 

In [26]:
import xlsxwriter
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

#### Creating the Formats We will Need For Our .xlsx File

Formats include colors, fonts, and also symbols like % and $. We will need four main formats for our Excel document:

 - String format for tickers
 - XX.XX format for stock prices
 - $XX.XXX format for market capitalization
 - Integer format for the number of shares to buy

In [27]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00', 
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [28]:
#### Applying the Formats to the Columns of Our .xlsx File

"""We can use the **set_column** method applied to the **writer.book** object to apply formats to specific columns of our spreadsheets.

Here's an example:

        writer.sheets['Momentum Strategy'].set_column('B:B', # This tells the method to apply the format to col B,
                                                        18, # This tells the method to apply a column width of 18 pixels
                                                        string_template # This applies the format 'string_template' to the column
                                                    )
                                                    
"""                                                    

"We can use the **set_column** method applied to the **writer.book** object to apply formats to specific columns of our spreadsheets.\n\nHere's an example:\n\n        writer.sheets['Momentum Strategy'].set_column('B:B', # This tells the method to apply the format to col B,\n                                                        18, # This tells the method to apply a column width of 18 pixels\n                                                        string_template # This applies the format 'string_template' to the column\n                                                    )\n                                                    \n"

In [29]:
# Formatting column values
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['One-Year Price Return', percent_format],
    'D': ['One-Year Return Percentile', percent_format],
    'E': ['Six-Month Price Return', percent_format],
    'F': ['Six-Month Return Percentile', percent_format],
    'G': ['Three-Month Price Return', percent_format],
    'H': ['Three-Month Return Percentile', percent_format],
    'I': ['One-Month Price Return', percent_format],
    'J': ['One-Month Return Percentile', percent_format],
    'K': ['Number of Shares to Buy', integer_format],
    'L': ['HQM Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], column_formats[column][1])

#### Saving Our Excel Output  

In [30]:
writer.close()